In [1]:
import sqlite3
import os
import re
from tkinter import Tk
from tkinter.filedialog import askdirectory

# Prompt user to select the folder to store the participant-specific database
def select_database_folder():
    root = Tk()
    root.withdraw()  # Hide root window
    root.attributes('-topmost', True)  # Bring dialog to the front
    folder = askdirectory(initialdir='D:/Action+/Data/', title="Select Database Storage Folder")
    root.destroy()
    return folder

# Prompt user to select the folder for the participant-specific database
db_folder = select_database_folder()
if not db_folder:
    raise ValueError("No folder selected for the database. Cannot proceed.")

# Define paths
data_folder = "D:/Action+/Exports"
participant_db_path = os.path.join(db_folder, 'baseball_database.db')
mass_db_path = "D:/Action+/Action+Database.db"

# Ensure the participant database folder exists
os.makedirs(db_folder, exist_ok=True)

# Delete the participant-specific database if it exists
if os.path.exists(participant_db_path):
    os.remove(participant_db_path)
    print(f"Deleted existing database at {participant_db_path}")

# Connect to SQLite databases
participant_conn = sqlite3.connect(participant_db_path)
mass_conn = sqlite3.connect(mass_db_path)
participant_cursor = participant_conn.cursor()
mass_cursor = mass_conn.cursor()

# Define table schemas
table_schemas = {
    'pitch': '''CREATE TABLE IF NOT EXISTS Pitch (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT,
                    trial_name TEXT,
                    Arm_Abduction_FP REAL,
                    Max_Abduction REAL,
                    Max_ER REAL,
                    Arm_Velo REAL,
                    Max_Torso_Rot_Velo REAL
                )''',
    'shortstop': '''CREATE TABLE IF NOT EXISTS Shortstop (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT,
                        trial_name TEXT,
                        Arm_Abduction_FP REAL,
                        Max_Abduction REAL,
                        Max_ER REAL,
                        Arm_Velo REAL,
                        Max_Torso_Rot_Velo REAL
                    )''',
    'catchers': '''CREATE TABLE IF NOT EXISTS Catchers (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT,
                        trial_name TEXT,
                        Arm_Abduction_FP REAL,
                        Max_Abduction REAL,
                        Max_ER REAL,
                        Arm_Velo REAL,
                        Max_Torso_Rot_Velo REAL
                    )''',
}

# Create tables in both databases
for table_name, schema in table_schemas.items():
    participant_cursor.execute(schema)
    mass_cursor.execute(schema)
    print(f"Table '{table_name.title()}' created successfully in both databases.")

# Function to process and insert data into both databases
def process_file(file_path, table_name):
    with open(file_path, 'r') as f:
        lines = f.readlines()

        # Extract participant name from the first line
        name_match = re.search(r'Data\\(.*?)[_\\]', lines[0])
        name = name_match.group(1).strip() if name_match else "Unknown"

        # Extract headers from line 2
        headers = lines[1].strip().split("\t")
        valid_headers = [
            "Arm_Abduction@Footplant",
            "Max_Abduction",
            "Max_ER",
            "Arm_Velo",
            "Max_Torso_Rot_Velo"
        ]

        if headers[:5] != valid_headers:
            print(f"Header mismatch in file {file_path}. Skipping.")
            return

        # Process the sixth line for trial data
        data_line = lines[5].strip().split()
        if len(data_line) < 6:  # Ensure there's enough data
            print(f"Insufficient data in {file_path}. Skipping.")
            return

        # Skip the first value ('1') and process in chunks of 5
        trials = data_line[1:]  # Skip the first "1"
        for i in range(0, len(trials), 5):
            chunk = trials[i:i + 5]
            if len(chunk) == 5:  # Only process complete groups of 5
                try:
                    variables = [float(value) for value in chunk]
                    trial_name = os.path.splitext(os.path.basename(file_path))[0]
                    participant_cursor.execute(f"""
                        INSERT INTO {table_name} (name, trial_name, Arm_Abduction_FP, Max_Abduction, Max_ER, Arm_Velo, Max_Torso_Rot_Velo)
                        VALUES (?, ?, ?, ?, ?, ?, ?)""", (name, trial_name, *variables))
                    mass_cursor.execute(f"""
                        INSERT INTO {table_name} (name, trial_name, Arm_Abduction_FP, Max_Abduction, Max_ER, Arm_Velo, Max_Torso_Rot_Velo)
                        VALUES (?, ?, ?, ?, ?, ?, ?)""", (name, trial_name, *variables))
                except ValueError as e:
                    print(f"Error processing trial data in file {file_path}: {chunk}. Error: {e}")

# Loop through `.txt` files and process
for file_name in os.listdir(data_folder):
    file_path = os.path.join(data_folder, file_name)
    file_name_lower = file_name.lower()

    # Determine the appropriate table
    if 'pitch' in file_name_lower:
        process_file(file_path, 'pitch')
    elif 'shortstop' in file_name_lower:
        process_file(file_path, 'shortstop')
    elif 'catchers' in file_name_lower:
        process_file(file_path, 'catchers')
    else:
        print(f"Skipped file: {file_name} (Unknown table)")

# Commit changes and close connections
participant_conn.commit()
participant_conn.close()
mass_conn.commit()
mass_conn.close()

print(f"Databases successfully populated. Participant-specific database saved at {participant_db_path}.")
print(f"Mass database updated and saved at {mass_db_path}.")


Deleted existing database at D:/Action+/Data/Gunter, Mason_MG/2024-12-03_\baseball_database.db
Table 'Pitch' created successfully in both databases.
Table 'Shortstop' created successfully in both databases.
Table 'Catchers' created successfully in both databases.
Databases successfully populated. Participant-specific database saved at D:/Action+/Data/Gunter, Mason_MG/2024-12-03_\baseball_database.db.
Mass database updated and saved at D:/Action+/Action+Database.db.


In [1]:
import sqlite3
import os
import re
from tkinter import Tk
from tkinter.filedialog import askdirectory

# Prompt user to select the folder to store the database
def select_database_folder():
    root = Tk()
    root.withdraw()  # Hide root window
    root.attributes('-topmost', True)  # Bring dialog to the front
    folder = askdirectory(initialdir='D:/Action+/Data/', title="Select Database Storage Folder")
    root.destroy()
    return folder

# Prompt user to select the folder for the database
db_folder = select_database_folder()
if not db_folder:
    raise ValueError("No folder selected for the database. Cannot proceed.")

# Define paths
data_folder = "D:/Action+/Exports"
db_path = os.path.join(db_folder, 'baseball_database.db')

# Ensure the database folder exists
os.makedirs(db_folder, exist_ok=True)

# Delete the database file if it exists
if os.path.exists(db_path):
    os.remove(db_path)
    print(f"Deleted existing database at {db_path}")

# Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Define the Pitch table schema (matching the column headers from the txt file)
table_schema = '''CREATE TABLE IF NOT EXISTS Pitch (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT,
                    trial_name TEXT,
                    Arm_Abduction_FP REAL,
                    Max_Abduction REAL,
                    Max_ER REAL,
                    Arm_Velo REAL,
                    Max_Torso_Rot_Velo REAL
                )'''
cursor.execute(table_schema)
print("Table 'Pitch' created successfully.")

# Function to process and insert data
def process_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

        # Extract participant name from the first line
        name_match = re.search(r'Data\\(.*?)[_\\]', lines[0])
        name = name_match.group(1).strip() if name_match else "Unknown"

        # Extract headers from line 2 (for verification)
        headers = lines[1].strip().split("\t")
        expected_headers = [
            "Arm_Abduction@Footplant",
            "Max_Abduction",
            "Max_ER",
            "Arm_Velo",
            "Max_Torso_Rot_Velo"
        ]
        if headers[:5] != expected_headers:
            print(f"Header mismatch in file {file_path}. Skipping.")
            return

        # Process data starting from line 6
        for line in lines[5:]:
            values = line.split()
            if len(values) >= 6:  # Ensure enough columns (including the extraneous '1')
                try:
                    # Skip the first value ('1') and parse the remaining columns
                    variables = [float(value) for value in values[1:6]]  # Extract 5 numeric columns
                    trial_name = os.path.splitext(os.path.basename(file_path))[0]
                    cursor.execute(f"""
                        INSERT INTO Pitch (name, trial_name, Arm_Abduction_FP, Max_Abduction, Max_ER, Arm_Velo, Max_Torso_Rot_Velo)
                        VALUES (?, ?, ?, ?, ?, ?, ?)""", (name, trial_name, *variables))
                except ValueError as e:
                    print(f"Error processing line in file {file_path}: {line}. Error: {e}")

# Loop through `.txt` files and process
for file_name in os.listdir(data_folder):
    if file_name.endswith('.txt'):
        file_path = os.path.join(data_folder, file_name)
        process_file(file_path)

# Commit changes and close connection
conn.commit()
conn.close()

print(f"Database successfully populated and saved at {db_path}")


Deleted existing database at D:/Action+/Data/Gunter, Mason_MG/2024-12-03_\baseball_database.db
Table 'Pitch' created successfully.
Database successfully populated and saved at D:/Action+/Data/Gunter, Mason_MG/2024-12-03_\baseball_database.db
